In [1]:
%matplotlib inline
import numpy as np
#pip install dill
#pip install surmise
import os
from os import path
import sys
import dill
import pickle
sys.path.insert(0, path.abspath('./'))

from src.emulator_BAND import EmulatorBAND
#from src.emulator import Emulator

parent="../actual"


/home/goetz/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Define function to create pickle file with training dataset and validation test points

Use the last 5 parameters as validation. Later we might add posterior points.

In [2]:
def read_config(file):
    with open(file, 'r') as f:
        # Initialize an empty dictionary to store the values
        values = []
        # Read the file line by line
        for line in f:
            # If the line starts with a '#', it's a comment
            if line.startswith('#'):
                # Split the line at the colon
                parts = line.split(':')
                if len(parts) < 2:
                    continue
                # The key is the part before the colon, stripped of leading/trailing whitespace and the '#'
                key = parts[0].strip().lstrip('#')
                # The value is the part after the colon, stripped of leading/trailing whitespace
                value = float(parts[1].strip())
                # Add the key-value pair to the dictionary
                values.append(value)
    return values


import pandas as pd
import numpy as np

####################
#GLOBAL VARIABLES
num_points=500
#num_points=4
####################



#generate the data pickles
path_data = parent+'/merged_directory'
path_configs = parent+'/configs'
path_output = parent+'/latent_pickled/'
path_output_exp = parent+'/exp/'
path_input_exp=parent+'/exp_input/0'
datasets = [
    'exp_19.6_05_eta_spectra', 'exp_200_05_y_spectra_piminus',
    'exp_19.6_05_integrated', 'exp_200_05_y_spectra_piplus',
    'exp_19.6_05_pT_spectra_kminus', 'exp_200_1525_eta_spectra',
    'exp_19.6_05_pT_spectra_kplus', 'exp_200_1525_phobos_v2_spectra',
    'exp_19.6_05_pT_spectra_p', 'exp_200_2030_integrated',
    'exp_19.6_05_pT_spectra_pbar', 'exp_200_2030_phenix_pT_v2_spectra',
    'exp_19.6_05_pT_spectra_piminus', 'exp_200_2030_phenix_pT_v3_spectra',
    'exp_19.6_05_pT_spectra_piplus', 'exp_200_2030_pT_spectra_kminus',
    'exp_19.6_1525_eta_spectra', 'exp_200_2030_pT_spectra_kplus','exp_200_2030_pT_spectra_p',
    'exp_19.6_2030_integrated', 'exp_200_2030_pT_spectra_pbar',
    'exp_19.6_2030_pT_spectra_kminus', 'exp_200_2030_pT_spectra_piminus',
    'exp_19.6_2030_pT_spectra_kplus', 'exp_200_2030_pT_spectra_piplus',
    'exp_19.6_2030_pT_spectra_p', 'exp_7.7_05_integrated',
    'exp_19.6_2030_pT_spectra_pbar', 'exp_7.7_05_pT_spectra_kminus',
    'exp_19.6_2030_pT_spectra_piminus', 'exp_7.7_05_pT_spectra_kplus',
    'exp_19.6_2030_pT_spectra_piplus', 'exp_7.7_05_pT_spectra_p',
    'exp_19.6_2030_star_v2_pT_spectra', 'exp_7.7_05_pT_spectra_pbar',
    'exp_200_05_eta_spectra', 'exp_7.7_05_pT_spectra_piminus',
    'exp_200_05_integrated', 'exp_7.7_05_pT_spectra_piplus',
    'exp_200_05_pT_spectra_kminus', 'exp_7.7_2030_integrated',
    'exp_200_05_pT_spectra_kplus', 'exp_7.7_2030_pT_spectra_kminus',
    'exp_200_05_pT_spectra_p', 'exp_7.7_2030_pT_spectra_kplus',
    'exp_200_05_pT_spectra_pbar', 'exp_7.7_2030_pT_spectra_p',
    'exp_200_05_pT_spectra_piminus', 'exp_7.7_2030_pT_spectra_pbar',
    'exp_200_05_pT_spectra_piplus', 'exp_7.7_2030_pT_spectra_piminus',
    'exp_200_05_y_spectra_kminus', 'exp_7.7_2030_pT_spectra_piplus',
    'exp_200_05_y_spectra_kplus', 'exp_7.7_2030_star_v2_pT_spectra'
]
integrated_values=[
    "dNdy_kminus",
    "dNdy_kplus",
    "dNdy_p",
    "dNdy_pbar",
    "dNdy_piminus",
    "dNdy_piplus",
    "meanpT_kminus",
    "meanpT_kplus",
    "meanpT_p",
    "meanpT_pbar",
    "meanpT_piminus",
    "meanpT_piplus",
    "star_v2",
    "star_v3"
]
 

import random 

# flags if certain sets should be used or not
def pickle_data_in_from_to(path_output, start, end,  outname,eta_cut=None,star_pT_cut=None, data="all", system=None, energy=None):
    #print("RAndom variation added!")
    dict_df_full={}
    datasets_local = datasets.copy()
    if system is not None:
        datasets_local = [d for d in datasets if system in d]
    if energy is not None:
        datasets_local = [d for d in datasets_local if energy in d]
    match data:
        case "base_wo_phobos":
            datasets_local = [d for d in datasets_local if ("phobos" not in d and "eta" not in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_200_only":
            datasets_local = [d for d in datasets_local if ("200" in d and "phobos" not in d and "eta" not in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_200_and_phobos":
            datasets_local = [d for d in datasets_local if ("200" in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base":
           datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_and_pT_spectra":
           datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)] 
        case "base_and_y_spectra":
            datasets_local = [d for d in datasets_local if ( "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_and_phenix":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d  and "star_v2_pT_spectra" not in d)]
        case "base_and_v2pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d)  or ( "star_v2_pT_spectra" in d )]
        case "base_and_phenix_and_v2pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d ) or ( "star_v2_pT_spectra" in d )]
        case "base_and_phenix_and_pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "star_v2_pT_spectra" not in d )]
        case "base_and_v2pT_and_pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "phenix" not in d )]
        case "base_and_all_but_y":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d )]
        case "base_and_y_and_pT_spectra":
            datasets_local = [d for d in datasets_local if ("phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "all":
            pass
        case _:
            raise ValueError("Unexpected case encountered")
    for i in range(start, end):
        i_str = str(i+1).zfill(3)
        dict_df={"obs": [], "parameter": [], "name":[], "lim":[0]}
        config_path = path_configs +"/config_sets_run_"+str(i+1).zfill(3)+".yaml"
        dict_df["parameter"]=read_config(config_path)
        obs_1 = []  # List for first entries of tuples
        obs_2 = []  # List for second entries of tuples
        for dataset in datasets_local:
            if "integrated" not in dataset:
                current_path = path_data +"/"+i_str+"/"+dataset.replace("exp", i_str)
                
                df = pd.read_csv(current_path)
                if eta_cut is not None and ("eta" in dataset or "y_spectra" in dataset or "phobos" in dataset):
                    df_filtered = df[abs(df.iloc[:, -3]) < eta_cut]
                    new_obs_1 = df_filtered.iloc[:, -2].tolist()
                    new_obs_2 = df_filtered.iloc[:, -1].tolist()
                    obs_1.extend(new_obs_1)
                    obs_2.extend(new_obs_2)
                    dict_df["lim"].append(df_filtered.shape[0] + dict_df["lim"][-1])
                elif star_pT_cut is not None and "star_v2_pT_spectra" in dataset:
                    df_filtered = df[abs(df.iloc[:, -3]) < star_pT_cut]
                    new_obs_1 = df_filtered.iloc[:, -2].tolist()
                    new_obs_2 = df_filtered.iloc[:, -1].tolist()
                    obs_1.extend(new_obs_1)
                    obs_2.extend(new_obs_2)
                    dict_df["lim"].append(df_filtered.shape[0] + dict_df["lim"][-1])
                else:
                    new_obs_1 = df.iloc[:, -2].tolist()
                    new_obs_2 = df.iloc[:, -1].tolist()
                    obs_1.extend(new_obs_1)
                    obs_2.extend(new_obs_2)
                    dict_df["lim"].append(df.shape[0] + dict_df["lim"][-1])

                # Check for NaNs in the newly added part of obs_1 and print a warning if any are found
                if any(np.isnan(new_obs_1)):
                    print(f"Warning: NaN found in obs_1 for dataset {dataset} with parameter {dict_df['parameter']}")
                dict_df["name"].append(dataset)
                
            elif "integrated" in dataset:
                for key in integrated_values:
                    current_path = path_data +"/"+i_str+"/"+dataset.replace("exp", i_str)   
                    df = pd.read_csv(current_path)
                    obs_1.extend(df[key].values*random.uniform(0.8, 2))
                    obs_2.extend(df[key+"_error"].values)
                    dict_df["name"].append(dataset+"_"+key)
                    dict_df["lim"].append(df.shape[0]+dict_df["lim"][-1])
        dict_df["obs"] = np.vstack((obs_1, obs_2))
        dict_df_full[i]=dict_df
    with open( path_output+outname+".pkl", 'wb') as f:
        pickle.dump(dict_df_full, f)
        
def pickle_data_exp(path_output, outname,eta_cut=None,star_pT_cut=None,data="all", system=None, energy=None):
    dict_df_full={}
    dict_df={"obs": [],  "name":[], "lim":[0]}
    obs_1 = []  # List for first entries of tuples
    obs_2 = []  # List for second entries of tuples
    datasets_local = datasets.copy()
    if system is not None:
        datasets_local = [d for d in datasets if system in d]
    if energy is not None:
        datasets_local = [d for d in datasets_local if energy in d]
    match data:
        case "base_wo_phobos":
            datasets_local = [d for d in datasets_local if ("phobos" not in d and "eta" not in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_200_only":
            datasets_local = [d for d in datasets_local if ("200" in d and "phobos" not in d and "eta" not in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_200_and_phobos":
            datasets_local = [d for d in datasets_local if ("200" in d and "y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_and_pT_spectra":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)] 
        case "base_and_y_spectra":
            datasets_local = [d for d in datasets_local if ( "pT_spectra" not in d and "phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "base_and_phenix":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d  and "star_v2_pT_spectra" not in d)]
        case "base_and_v2pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d and "phenix" not in d) or ( "star_v2_pT_spectra" in d )]
        case "base_and_phenix_and_v2pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "pT_spectra" not in d )  or ( "star_v2_pT_spectra" in d )]
        case "base_and_phenix_and_pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "star_v2_pT_spectra" not in d )]
        case "base_and_v2pT_and_pT":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d and "phenix" not in d )]
        case "base_and_all_but_y":
            datasets_local = [d for d in datasets_local if ("y_spectra" not in d )]
        case "base_and_y_and_pT_spectra":
            datasets_local = [d for d in datasets_local if ("phenix" not in d and "star_v2_pT_spectra" not in d)]
        case "all":
            pass
        case _:
            raise ValueError("Unexpected case encountered")
    for dataset in datasets_local:
        if "integrated" not in dataset:
            current_path = path_input_exp+"/"+dataset
            df = pd.read_csv(current_path)
            if eta_cut is not None and ("eta" in dataset or "y_spectra" in dataset or "phobos" in dataset):
                df_filtered = df[abs(df.iloc[:, -3]) < eta_cut]
                obs_1.extend(df_filtered.iloc[:, -2])
                obs_2.extend(df_filtered.iloc[:, -1])
                dict_df["lim"].append(df_filtered.shape[0]+dict_df["lim"][-1])
            elif star_pT_cut is not None and "star_v2_pT_spectra" in dataset:
                df_filtered = df[abs(df.iloc[:, -3]) < star_pT_cut]
                obs_1.extend(df_filtered.iloc[:, -2])
                obs_2.extend(df_filtered.iloc[:, -1])
                dict_df["lim"].append(df_filtered.shape[0]+dict_df["lim"][-1])
            else:
                obs_1.extend(df.iloc[:, -2])
                obs_2.extend(df.iloc[:, -1])
            
            dict_df["name"].append(dataset)
            dict_df["lim"].append(df.shape[0]+dict_df["lim"][-1])
        elif "integrated" in dataset:
            for key in integrated_values:
                current_path = path_input_exp+"/"+dataset   
                df = pd.read_csv(current_path)
                obs_1.extend(df[key].values)
                obs_2.extend(df[key+"_error"].values)
                dict_df["name"].append(dataset+"_"+key)
                dict_df["lim"].append(df.shape[0]+dict_df["lim"][-1])
    dict_df["obs"] = np.vstack((obs_1, obs_2))
    dict_df_full[0]=dict_df
    with open( path_output+outname+".pkl", 'wb') as f:
        pickle.dump(dict_df_full, f)
pickle_data_exp(path_output_exp,  "exp")



[INFO][utils] NumExpr defaulting to 12 threads.


In [3]:
#do they have to be pickels?
#the pickels contain dictonaries. the dictionary is [{obs: [[val1, val2...], [err1,err2...]], parameter: i}] with length design points
"""
dataDict = {
    'event_id1': { #id of parameter points
        'obs': [[...], [...]],  # 2D array-like structure, values and errors
        'parameter': ...  # parameter list, just pure numerical values
    },
    'event_id2': {
        'obs': [[...], [...]],  # 2D array-like structure
        'parameter': ...  # Scalar or 1D array
    },
    ...
}
"""
#print("The following pickles have been created:")
#datasets = list_of_pickles
path_output_train = parent+'/latent_train/'
path_output_val = parent+'/latent_val/'
# Check if the output folder exists, if not, create it
if not os.path.exists(path_output_train):
    os.makedirs(path_output_train)

if not os.path.exists(path_output_val ):
    os.makedirs(path_output_val )

def check_file_length(filename, expected_length):
    with open(f"{path_output}{filename}", 'rb') as f:
        data = pickle.load(f)
    if len(data) == expected_length:
        pass
    else:
        print(f"{filename} does not have the correct length. Expected: {expected_length}, Actual: {len(data)}")

# datasets_train = []
# for dataset in datasets:
#     current_path = dataset
#     #print(dataset)
#     with open(current_path, "rb") as pf:
#         data = pickle.load(pf)
    
#     #print(f"The dataset {dataset} has length {len(data)}.")
#     # Separate data based on event ID
#     sorted_event_ids = sorted(data.keys(), key=lambda x: int(x))
#     first_data = {event_id: data[event_id] for event_id in sorted_event_ids[:num_points-1]}
#     second_data = {event_id: data[event_id] for event_id in sorted_event_ids[num_points-1:]}
#     #print(f'{dataset.split(".p")[0].split("/")[-1]}_train.pkl')
#     # Save separated data to pickle files
#     with open(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl', 'wb') as pf1:
#         pickle.dump(first_data, pf1)
        
#     with open(f'{path_output_val}{dataset.split(".p")[0].split("/")[-1]}_val.pkl', 'wb') as pf2:
#         pickle.dump(second_data, pf2)

#     check_file_length(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl', num_points-1)
#     datasets_train.append(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl')
#     check_file_length(f'{path_output_val}{dataset.split(".p")[0].split("/")[-1]}_val.pkl', 1) 

pickle_data_in_from_to(path_output_train, 0, num_points-15, "train_all_pt08", data="all",star_pT_cut=0.8)
pickle_data_in_from_to(path_output_train, 0, num_points, "full_all_pt08", data="all",star_pT_cut=0.8)
pickle_data_in_from_to(path_output_val, num_points-15, num_points, "val_all_pt08", data="all",star_pT_cut=0.8)

# Perform the training of the emulators for all datasets

After training the emulators, we save them with `dill`, such that they can be reloaded from file for the MCMC later.

Perform the training without the `parameterTrafoPCA` (for $\zeta/s(T)$, $\eta/s(\mu_B)$, $\langle y_{\rm loss}\rangle(y_{\rm init})$).

In [4]:
#model_par contains the full range of values for the parameters
model_par = parent+'/config_AuAu_200_bulk_scan_central.yaml'
path_input =parent+'/latent_train/'
path_output = parent+'/trained_emulators_no_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)
emu1 = EmulatorBAND(f'{path_input}full_all_pt08.pkl', model_par, method='PCGP', logTrafo=False, parameterTrafoPCA=False)

emu1.trainEmulatorAutoMask()
# emu2 = EmulatorBAND(f'{path_input}full_base_cut_3.pkl', model_par, method='PCSK', logTrafo=False, parameterTrafoPCA=False)
# emu2.trainEmulatorAutoMask()
with open(f'{path_output}PCGP_trained_full_all_pt08.sav', 'wb') as f:
    dill.dump(emu1, f)
# with open(f'{path_output}PCSK_trained_full_base_cut_3.sav', 'wb') as f:
#     dill.dump(emu2, f)



[INFO][emulator_BAND] loading training data from ../actual/latent_train/full_all_pt08.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 500, discarded points: 0
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train GP emulators with 500 training points ...
[[0.101052   0.229098   0.107692   ... 0.077978   0.157804   0.061668  ]
 [0.150238   0.342718   0.16244    ... 0.116494   0.234736   0.091806  ]
 [0.224898   0.508262   0.242304   ... 0.175324   0.351114   0.138622  ]
 ...
 [0.02411828 0.02286398 0.02659007 ... 0.02223427 0.02585007 0.0251167 ]
 [0.04093649 0.0352128  0.04575192 ... 0.03851581 0.04444132 0.04276902]
 [0.05073053 0.0424563  0.0604417  ... 0.04958737 0.05825397 0.05349481]]
29.270631519181244
73.1091858662999


OSError: [Errno 28] No space left on device

Perform the training with the `parameterTrafoPCA` (for $\zeta/s(T)$, $\eta/s(\mu_B)$, $\langle y_{\rm loss}\rangle(y_{\rm init})$).

In [3]:
model_par = parent+'/config_AuAu_200_bulk_scan_central.yaml'
path_input =parent+'/latent_train/'
path_output = parent+'/trained_emulators_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

emu1 = EmulatorBAND(f'{path_input}full_base_cut_3.pkl', model_par, method='PCGP', logTrafo=False, parameterTrafoPCA=True)

emu1.trainEmulatorAutoMask()
emu2 = EmulatorBAND(f'{path_input}full_base_cut_3.pkl', model_par, method='PCSK', logTrafo=False, parameterTrafoPCA=True)
emu2.trainEmulatorAutoMask()
with open(f'{path_output}PCGP_trained_full_base_cut_3.sav', 'wb') as f:
    dill.dump(emu1, f)
with open(f'{path_output}PCSK_trained_full_base_cut_3.sav', 'wb') as f:
    dill.dump(emu2, f)



#: 2 and 5 PCA components 

[INFO][emulator_BAND] loading training data from ../actual/latent_train/full_base_cut_3.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 500, discarded points: 0
[INFO][emulator_BAND] Prepare bulk viscosity parameter PCA ...
[[4.97508497e-009 1.28246332e-005 4.24023592e-005 ... 2.73545395e-002
  2.73291544e-002 2.73037948e-002]
 [6.71945085e-005 1.01456398e-003 1.56951307e-003 ... 5.16212146e-002
  5.16012634e-002 5.15813109e-002]
 [1.89498810e-002 6.58729047e-002 7.89953924e-002 ... 5.21947278e-002
  5.21200169e-002 5.20454275e-002]
 ...
 [1.16744677e-007 6.32821507e-005 1.69316116e-004 ... 1.20087071e-001
  1.20026159e-001 1.19965274e-001]
 [4.42333800e-004 5.44360355e-003 8.08772161e-003 ... 1.00593013e-001
  1.00519520e-001 1.00446069e-001]
 [0.00000000e+000 4.21611723e-194 1.08803700e-170 ... 2.59221471e-002
  2.59080526e-00

In [ ]:
model_par = parent+'/config_AuAu_200_bulk_scan_central.yaml'
path_input =parent+'/latent_train/'
path_output = parent+'/trained_emulators_LOG/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

emu1 = EmulatorBAND(f'{path_input}full_base_cut_3.pkl', model_par, method='PCGP', logTrafo=True, parameterTrafoPCA=False)

emu1.trainEmulatorAutoMask()
emu2 = EmulatorBAND(f'{path_input}full_base_cut_3.pkl', model_par, method='PCSK', logTrafo=True, parameterTrafoPCA=False)
emu2.trainEmulatorAutoMask()
with open(f'{path_output}PCGP_trained_full_base_cut_3.sav', 'wb') as f:
    dill.dump(emu1, f)
with open(f'{path_output}PCSK_trained_full_base_cut_3.sav', 'wb') as f:
    dill.dump(emu2, f)
    


[INFO][emulator_BAND] loading training data from ../latent_train/train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 4, discarded points: 0
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train GP emulators with 4 training points ...
[INFO][emulator_BAND] loading training data from ../latent_train/train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 4, discarded points: 0
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train GP emulators with 4 training points ...


In [ ]:
model_par = parent+'/config_AuAu_200_bulk_scan_central.yaml'
path_input =parent+'/latent_train/'
path_output = parent+'/trained_emulators_LOG_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

if not os.path.exists(path_output):
    os.makedirs(path_output)

emu1 = EmulatorBAND(f'{path_input}full.pkl', model_par, method='PCGP', logTrafo=True, parameterTrafoPCA=True)

emu1.trainEmulatorAutoMask()
emu2 = EmulatorBAND(f'{path_input}full.pkl', model_par, method='PCSK', logTrafo=True, parameterTrafoPCA=True)
emu2.trainEmulatorAutoMask()
with open(f'{path_output}PCGP_trained_full.sav', 'wb') as f:
    dill.dump(emu1, f)
with open(f'{path_output}PCSK_trained_full.sav', 'wb') as f:
    dill.dump(emu2, f)

[INFO][emulator_BAND] loading training data from ../latent_train/train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 4, discarded points: 0
[INFO][emulator_BAND] Prepare bulk viscosity parameter PCA ...
[INFO][emulator_BAND] Bulk viscosity parameter PCA uses 3 PCs to explain 99.0% of the variance ...
[INFO][emulator_BAND] Prepare shear viscosity parameter PCA ...
[INFO][emulator_BAND] Shear viscosity parameter PCA uses 1 PCs to explain 99.0% of the variance ...
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train GP emulators with 4 training points ...
[INFO][emulator_BAND] loading training data from ../latent_train/train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator

Generate an 'experimental' dataset from one of the posterior points for closure testing.

In [3]:
# path_data = './separate_training_posterior_data_1095/'
# path_output = './separate_training_posterior_data_1095/'
# datasets_posterior = [
#             'AuAu200_dNdy_posterior.pkl',
#             'AuAu200_PHOBOSdNdeta_posterior.pkl',
#             'AuAu200_PHOBOSv2eta_posterior.pkl',
#             'AuAu200_pTvn_posterior.pkl',
#             'AuAu19p6_dNdy_posterior.pkl',
#             'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
#             'AuAu19p6_pTvn_posterior.pkl',
#             'AuAu7.7_dNdy_posterior.pkl',
#             'AuAu7.7_pTvn_posterior.pkl']

# # Check if the output folder exists, if not, create it
# if not os.path.exists(path_output):
#     os.makedirs(path_output)

# event_data = []
# for dataset in datasets_posterior:
#     current_path = path_data + dataset
#     with open(current_path, "rb") as pf:
#         data = pickle.load(pf)

#     # Get the first event from the posterior dataset
#     test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[4:5]}
#     event_data.append(test_data)


# for event_dict in event_data[1:]:
#     # Get the 'obs' array for the current event
#     obs_array_new = event_dict['1099']['obs']
    
#     # Extend the 'obs' array of the first element with the values from the current event
#     event_data[0]['1099']['obs'] = np.concatenate((event_data[0]['1099']['obs'], obs_array_new), axis=1)

# # Save separated data to pickle files
# with open(f'{path_output}example_data_test_point1099.pkl', 'wb') as pf1:
#     pickle.dump(event_data[0], pf1)


In [2]:
# path_data = './separate_training_posterior_data_1095/'
# path_output = './separate_training_posterior_data_1095/'
# datasets_posterior = [
#             'AuAu200_dNdy_posterior.pkl',
#             'AuAu200_PHOBOSdNdeta_posterior.pkl',
#             'AuAu200_PHOBOSv2eta_posterior.pkl',
#             'AuAu200_pTvn_posterior.pkl',
#             'AuAu19p6_dNdy_posterior.pkl',
#             'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
#             'AuAu19p6_pTvn_posterior.pkl',
#             'AuAu7.7_dNdy_posterior.pkl',
#             'AuAu7.7_pTvn_posterior.pkl']

# # Check if the output folder exists, if not, create it
# if not os.path.exists(path_output):
#     os.makedirs(path_output)

# event_data = []
# for dataset in datasets_posterior:
#     current_path = path_data + dataset
#     with open(current_path, "rb") as pf:
#         data = pickle.load(pf)

#     # Get the first event from the posterior dataset
#     test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[2:3]}
#     event_data.append(test_data)


# for event_dict in event_data[1:]:
#     # Get the 'obs' array for the current event
#     obs_array_new = event_dict['1097']['obs']
    
#     # Extend the 'obs' array of the first element with the values from the current event
#     event_data[0]['1097']['obs'] = np.concatenate((event_data[0]['1097']['obs'], obs_array_new), axis=1)

# # Save separated data to pickle files
# with open(f'{path_output}example_data_test_point1097.pkl', 'wb') as pf1:
#     pickle.dump(event_data[0], pf1)


Generate the point for the test of the logarithmic training

In [5]:
path_data = './separate_training_posterior_data_1095/'
path_output = './separate_training_posterior_data_1095/'
datasets_posterior = [
            'AuAu200_dNdy_posterior.pkl',
            'AuAu200_PHOBOSdNdeta_posterior.pkl',
            'AuAu200_PHOBOSv2eta_posterior.pkl',
            'AuAu200_pTvn_posterior.pkl',
            'AuAu19p6_dNdy_posterior.pkl',
            'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
            'AuAu19p6_pTvn_posterior.pkl',
            'AuAu7.7_dNdy_posterior.pkl',
            'AuAu7.7_pTvn_posterior.pkl']

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

event_data = []
for dataset in datasets_posterior:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    # Get the first event from the posterior dataset
    test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[4:5]}
    event_data.append(test_data)

# modify the datasets with the multiplicities and take the log
datasets_to_modify = [0,1,4,5,7]
for i in datasets_to_modify:
    event_data[i]['1099']['obs'][0,:] = np.log(np.abs(event_data[i]['1099']['obs'][0,:]) + 1e-30)
    event_data[i]['1099']['obs'][1,:] = np.abs(event_data[i]['1099']['obs'][1,:]/event_data[i]['1099']['obs'][0,:] + 1e-30)

for event_dict in event_data[1:]:
    # Get the 'obs' array for the current event
    obs_array_new = event_dict['1099']['obs']
    
    # Extend the 'obs' array of the first element with the values from the current event
    event_data[0]['1099']['obs'] = np.concatenate((event_data[0]['1099']['obs'], obs_array_new), axis=1)

# Save separated data to pickle files
with open(f'{path_output}example_data_test_point1099_LOG.pkl', 'wb') as pf1:
    pickle.dump(event_data[0], pf1)

Generate one dataset from all of the training and posterior points.

In [8]:
path_data = '../data/'
path_output = './full_data_one_pkl/'
datasets_posterior = [
            'AuAu200_dNdy.pkl',
            'AuAu200_PHOBOSdNdeta.pkl',
            'AuAu200_PHOBOSv2eta.pkl',
            'AuAu200_pTvn.pkl',
            'AuAu19p6_dNdy.pkl',
            'AuAu19p6_PHOBOSdNdeta.pkl',
            'AuAu19p6_pTvn.pkl',
            'AuAu7.7_dNdy.pkl',
            'AuAu7.7_pTvn.pkl']

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

event_data = []
for dataset in datasets_posterior:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    # Get the first event from the posterior dataset
    sorted_keys = sorted(data.keys())
    test_data = {event_id: data[event_id] for event_id in sorted_keys}
    event_data.append(test_data)

for dataset in event_data[1:]:
    for event in sorted_keys:
        # Get the 'obs' array for the current event
        obs_array_new = dataset[event]['obs']
        
        # Extend the 'obs' array of the first dataset with the values from the others
        event_data[0][event]['obs'] = np.concatenate((event_data[0][event]['obs'], obs_array_new), axis=1)

# Save separated data to pickle files
with open(f'{path_output}all_points_all_observables.pkl', 'wb') as pf1:
    pickle.dump(event_data[0], pf1)

# Delete parameters 16 and 17 from pkl files (bulk_max_rhob2,bulk_max_rhob4)

In [ ]:
path_data = '../data/'
path_output = '../data_new/'
datasets = ['AuAu7.7_dNdy.pkl',
            'AuAu7.7_pTvn.pkl',
            'AuAu19p6_dNdy.pkl',
            'AuAu19p6_PHOBOSdNdeta.pkl',
            'AuAu19p6_pTvn.pkl',
            'AuAu200_dNdy.pkl',
            'AuAu200_PHOBOSdNdeta.pkl',
            'AuAu200_PHOBOSv2eta.pkl',
            'AuAu200_pTvn.pkl',
            'AuAu7.7_logdNdy.pkl',
            'AuAu19p6_logdNdy.pkl',
            'AuAu19p6_logPHOBOSdNdeta.pkl',
            'AuAu200_logdNdy.pkl',
            'AuAu200_logPHOBOSdNdeta.pkl'
            ]

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    print(f"The dataset {dataset} has length {len(data)}.")
    # Separate data based on event ID
    sorted_event_ids = sorted(data.keys(), key=lambda x: int(x))
    first_data = {event_id: data[event_id] for event_id in sorted_event_ids}
    print("Parameters before =",len(first_data[f'{sorted_event_ids[0]}']['parameter']))

    for point in range(len(sorted_event_ids)):
        first_data[f'{sorted_event_ids[point]}']['parameter'] = np.delete(first_data[f'{sorted_event_ids[point]}']['parameter'], [16,17])

    print("Parameters after =",len(first_data[f'{sorted_event_ids[0]}']['parameter']))
    # Save new data to pickle files
    with open(f'{path_output}{dataset.split(".p")[0]}.pkl', 'wb') as pf1:
        pickle.dump(first_data, pf1)

In [ ]:
# for dataset in datasets:
#     if "integrated" not in dataset:
#         dict_df_full={}
#         for i in range(num_points):
#             current_path = path_data +"/"+str(i)+"/"+dataset
#             config_path = path_configs +"/config_sets_run_"+str(i+1).zfill(3)+".yaml"
#             df = pd.read_csv(current_path)
#             dict_df={"obs":np.asarray([df.iloc[:, -2]*np.random.uniform(0.8, 2, df.shape[0])+np.random.uniform(0., 1e-10, df.shape[0]),df.iloc[:, -1]]), "parameter": read_config(config_path)}
#             #print(read_config(config_path))
#             dict_df_full[i]=dict_df
#         with open( path_output+dataset+".pkl", 'wb') as f:
#             pickle.dump(dict_df_full, f)
#         list_of_pickles.append(path_output+dataset+".pkl")
#     elif "integrated" in dataset:
#         for key in integrated_values:
#             dict_df_full={}
#             for i in range(num_points):
#                 current_path = path_data +"/"+str(i)+"/"+dataset
#                 config_path = path_configs +"/config_sets_run_"+str(i+1).zfill(3)+".yaml"
#                 df = pd.read_csv(current_path)
#                 dict_df={"obs":np.asarray([df[key].values*random.uniform(0.8, 2),df[key+"_error"].values]), "parameter": read_config(config_path)}
#                 dict_df_full[i]=dict_df
#             with open(path_output + dataset + "_" + key + ".pkl", 'wb') as f:
#                 pickle.dump(dict_df_full, f)
#             list_of_pickles.append(path_output+dataset+"_"+key+".pkl")
#print(list_of_pickles)